## Minimize variance without allowing short sales, no risk-aversion preference

<time>November 30, 2022</time>

In [1]:
# imports

import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp

Minimize 1/2 var subject to expected return = target

Different borrowing and savings rate

no short sales

Choose: xs, xb,w1, w2, w3

In [2]:
# given
mn1, mn2, mn3 = 0.1, 0.15, 0.12
sd1, sd2, sd3 = 0.2, 0.3, 0.4
corr12, corr13, corr23 = 0.1, 0.3, 0.5
rs = 0.03
rb = 0.05
target = 0.09

# get covariance matrix
S = np.diag([sd1, sd2, sd3])
R = np.identity(3)
R[0,1] = R[1,0] = corr12
R[0,2] = R[2,0] = corr13
R[1,2] = R[2,1] = corr23
C = S @ R @ S

In [3]:
# set up the P, q, G, h, A, b matrices
P = np.zeros((5,5))
P[2:, 2:] = C  # no risk-aversion because we are just trying to minimze variance, not optimal for target
q = np.zeros(5)

G = -np.identity(5)
# note, if we were allowing short sales, then G = -np.identity(2) as we drop the constraints on w1, w2, w3
h = np.zeros(5)
# allowing short sales, h = np.zeros(2)

A = np.array([[1., -1., 1., 1., 1.], [rs, -rb, mn1, mn2, mn3]])  # subject to expected return == target
b = np.array([[1], [target]])

In [4]:
# minimize variance

# Convert to cvxopt
P = matrix(P, (5,5))
q = matrix(q, (5,1))
G = matrix(G, (5,5))
h = matrix(h, (5,1))
A = matrix(A, (2,5))
b = matrix(b, (2,1))

# solve
sol = qp(P=P, q=q, G=G, h=h, A=A, b=b)

     pcost       dcost       gap    pres   dres
 0:  8.9067e-03 -5.0477e-01  7e+00  3e+00  3e+00
 1:  2.1078e-02 -5.8657e-01  8e-01  2e-01  2e-01
 2:  2.3818e-02 -7.0595e-03  3e-02  3e-15  1e-16
 3:  8.4305e-03  5.1340e-03  3e-03  9e-16  1e-17
 4:  7.2910e-03  6.8715e-03  4e-04  2e-16  6e-18
 5:  7.0326e-03  6.9988e-03  3e-05  1e-16  7e-18
 6:  7.0026e-03  7.0019e-03  7e-07  1e-15  7e-18
 7:  7.0020e-03  7.0020e-03  7e-09  2e-16  8e-18
Optimal solution found.


In [5]:
print(sol['x'])

[ 3.48e-01]
[ 1.65e-08]
[ 3.65e-01]
[ 2.87e-01]
[ 1.27e-06]



**To minimize variance without allowing short sales**:

Save: 34.8%

Borrow: 0%

Investment 1: 36.5%

Investment 2: 28.7%

Investment 3: 0%